In [ ]:
import cv2
import numpy as np
import glob

# ファイル一覧
files = glob.glob('recipt/*')
sigma = 0.33
for file in files:
    # 画像読み込み
    paper = cv2.imread(file)
    # グレースケール
    gray = cv2.cvtColor(paper, cv2.COLOR_BGR2GRAY)
    bit = cv2.bilateralFilter(gray, 9, 75, 75)
    two = cv2.adaptiveThreshold(bit, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 4)
    blur = cv2.medianBlur(two, 11)
    border = cv2.copyMakeBorder(blur, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    # 二値化
    v = np.median(border)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    canny = cv2.Canny(border, lower, upper)
    cv2.imwrite('mask'+file, canny)
    # 輪郭検出
    contours = cv2.findContours(canny, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
    contours.sort(key=cv2.contourArea, reverse=True)
    # 緑枠描写
    warp = None
    for con in contours:
        arclen = cv2.arcLength(con, True)
        approx = cv2.approxPolyDP(con, 0.02*arclen, True)
        if len(approx) == 4:
            cv2.drawContours(paper, [approx], -1, (0, 255, 0),5)
            break
    cv2.imwrite('done'+file, paper)